In [36]:
# IMPORTS AND LIBRARIES 
from qcodes.instrument_drivers.Keithley.Keithley_2614B import Keithley2614B
import qcodes as qc
from qcodes.station import Station
import qcodes.interactive_widget as qcw
from qcodes.instrument.parameter import Parameter
import qcodes.dataset as qcd
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_dataset
from qcodes.dataset.experiment_container import load_or_create_experiment

from qcodes import logger as qlogger

from pyvisa.errors import VisaIOError

import time
from IPython.display import display,clear_output
from IPython.core.display import HTML
from threading import Thread, Event

import ipywidgets as widgets
from ipywidgets import Layout
%matplotlib widget

import numpy as np

from IPython.core.getipython import get_ipython
import matplotlib.pyplot as plt

import sys
import io
import logging
import json
import os


from qcodes.dataset.sqlite.queries import get_experiment_name_from_experiment_id, get_last_experiment
from qcodes.dataset.experiment_container import load_last_experiment, load_experiment, experiments

In [ ]:
#GLOBAL VARIABLES AND PARAMETERS

sourcerange_dict_i = {
    "100nA":1e-7,
    "1μA":1e-6,
    "10μA":1e-5,
    "100μA":1e-4,
    "1mA":1e-3,
    "10mA":0.01,
    "100mA":0.1,
    "1A":1,
    "1.5A":1.5
}
sourcerange_dict_v={
    "200mV":0.2,
    "2V":2,
    "20V":20,
    "200V":200
}

cancel = False

db = None
output = widgets.Output()

channels_initialized = False

channel_A_hbox = widgets.HBox()

channel_B_hbox = widgets.HBox()

# station and database initialization 
station = Station()


class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to Jupyter's output widget """

    def __init__(self, widget):
        super().__init__()
        self.widget = widget

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        with self.widget:
            print(formatted_record)

# Create an instance of the OutputWidgetHandler
widget_handler = OutputWidgetHandler(output)

# Set a formatter if needed (optional)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
widget_handler.setFormatter(formatter)

logger = logging.getLogger('qcodes.dataset.measurements')  # Adjust this to the name of the logger you're interested in

# Add the handler to the logger
logger.addHandler(widget_handler)

# Adjust the log level to capture all messages (can be adjusted as needed)
logger.setLevel(logging.DEBUG)

with output : qlogger.start_all_logging()

USER_JSON_PATH = "users.json"
BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
USER_DATABASES_DIR = os.path.join(BASE_DIR, "UserDatabases")
USER_CONFIGS_DIR = os.path.join(BASE_DIR,"UserConfigs")
EXPORTS_DIR = os.path.join(BASE_DIR,"Exports")



# Check if the directory exists, if not, create it
if not os.path.exists(USER_DATABASES_DIR):
    os.makedirs(USER_DATABASES_DIR)
if not os.path.exists(USER_CONFIGS_DIR):
    os.makedirs(USER_CONFIGS_DIR)

def initialize_json_file():
    """
    Initializes the JSON file if it does not exist.
    """
    if not os.path.exists(USER_JSON_PATH):
        with open(USER_JSON_PATH, 'w') as json_file:
            json.dump({}, json_file)

def add_user(username):
    """
    Adds a new user to the JSON file.
    """
    config_path = os.path.join(USER_CONFIGS_DIR,f"{username}_config.json")
    db_path = os.path.join(USER_DATABASES_DIR, f"{username}_database.db")

    qc.config.core.db_location = db_path
    qc.config.dataset.export_automatic = True
    qc.config.dataset.export_type = "csv"
    qc.config.dataset.export_prefix = f"{username}_"    
    qc.config.save_config(config_path)


    with open(USER_JSON_PATH, 'r') as json_file:
        data = json.load(json_file)
    data[username] = {
        "db_path": db_path,
        "config_path": config_path
    }
    with open(USER_JSON_PATH, 'w') as json_file:
        json.dump(data, json_file)
    return db_path

def get_database_path(username):
    """
    Returns the database path for a given user.
    If the user does not exist, it will return None.
    """
    with open(USER_JSON_PATH, 'r') as json_file:
        data = json.load(json_file)
    user_data = data.get(username, None)
    if user_data:
        return user_data["db_path"]
    return None

initialize_json_file()

plot_output = widgets.Output()
# Initial empty data lists for x and y values
distances = []
resistances = []

# Create a figure and axis
fig, ax = plt.subplots(figsize=(6, 4))
line, = ax.plot([], [], 'o-')  # Line plot

# Function to update the plot
def update_plot():
    line.set_data(distances, resistances)
    ax.relim()
    ax.autoscale_view()
    with plot_output:
        clear_output(wait=True)
        plt.draw()
        plt.show(fig)

plot_box = widgets.VBox([plot_output])

# universal widgets : 
connect_btn = widgets.Button(
    description='Connect',
    button_style='',
    layout = widgets.Layout(width='100px',height='50px'),
    disabled = False
)
disconnect_btn = widgets.Button(
    description = 'Disconnect',
    button_style = 'danger',
    layout = widgets.Layout(width='100px',height='50px'),
    disabled = True
)
exit_btn = widgets.Button(
    description = 'EXIT',
    button_style = 'info',
    layout = Layout(width = '100px',height = '50px'),
    tooltip='EXIT key to back out from an error'
)
clear_btn = widgets.Button(
    description='Clear',
    button_style = 'info',
    layout = Layout(width = '100px',height='50px')
)

display_options = {
    "SMUA": 0,
    "SMUB": 1,
    "SMUA & SMUB": 2
}

display_dropdown = widgets.Dropdown(
    options = display_options.keys(),
    value = "SMUA"
)
display_dropdown.layout = Layout(width = '150px',height = '50px')
device_cd_hbox = widgets.Box([connect_btn,disconnect_btn,exit_btn,clear_btn,display_dropdown])
device_cd_hbox.layout = Layout(display='flex', margin='5px 0px 5px 0px')

dialog = widgets.VBox()
dialog.layout = widgets.Layout(
    padding='20px', 
    width='350px', 
    border='2px solid black',
    align_items = 'center'
)
output.layout = Layout(border='2px black solid', margin='20px 0px 20px 0px', height = '200px', overflow = 'auto')

output_box = widgets.Box([output,device_cd_hbox])
output_box.layout = Layout(display='flex',flex_flow='column')

In [38]:
#device functions 
def connect_device():
    try:
        with output:
            keith = Keithley2614B("keith", "ASRL3::INSTR")
            station.add_component(keith)
    except AttributeError:
        with output:
            print("The device component 'keith' is not properly added to the station.")
    except VisaIOError:
        with output:
            print("Could not connect to the device, please check if the device is powered on!")
    except Exception as e:
        with output:
            print(f"Device not available. Additional error info: {e}")
    else:
        with output:
            print(station.components)
            print("Connection successful.")
        return True
def disconnect_device():
    try:
        with output:
            station.keith.close()
            station.remove_component("keith")
    except:
        with output: print("No devices were disconnected!")
    else:
        with output: print("Devices successfully disconnected")
def fast_sweep(lower, upper, step, mode, smu, unit):
    try:
        steps = len(np.arange(lower,upper,step))
        if unit == 'nA':
            start = lower/1e9
            stop = upper/1e9
        elif unit == 'μA':
            start = lower/1e6
            stop = upper/1e6
        elif unit == 'mA':
            start = lower/1e3
            stop = upper/1e3
        elif unit == 'mV':
            start = lower/1e3
            stop = upper/1e3
        else:
            start = lower
            stop = upper
        smu.fastsweep.prepareSweep(
            start,
            stop,
            steps,
            mode
        )
        qcd.do0d(smu.fastsweep,do_plot=True)
    except Exception as e:
        with output:
            print(f"Error: {e}")
# recursive function for disabling/enabling widgets
def set_widget_disabled_status(container, status):
    if hasattr(container, 'children'):
        for child in container.children:
            set_widget_disabled_status(child, status)
    if hasattr(container, 'disabled'):
        container.disabled = status

In [39]:
# CHANNEL SETUP FUNCTION : 

def setup_channel(channel):

    if channel == 'A':
        smuX = station.keith.smua  
    else:
        smuX = station.keith.smub

    source_label = widgets.Label(f"SRC_{channel} :")
    source_range_dropdown = widgets.Dropdown(
        options = sourcerange_dict_v.keys(),
        value = "2V",
        layout = widgets.Layout(width='200px')
    )
    source_mode_dropdown = widgets.Dropdown(
        options = ["Voltage", "Current"],
        value = "Voltage",
        layout = widgets.Layout(width='100px')
    )

    sweep_label = widgets.Label(value='Sweep range :')
    unit_label = widgets.Label(value='')
    source_level_slider = widgets.FloatRangeSlider(
        value = (-0.1,0.1),
        min = -1,
        max = 1,
        descripton = ""
    )
    sweep_hbox = widgets.HBox([source_level_slider,unit_label])

    step_sel_label = widgets.Label(value = "Select step : ")
    step_selector = widgets.BoundedFloatText(
        value = 0.1,
        min = 0,
        max = 1,
        step = 0.01,
        description = "",
        layout = widgets.Layout(width='95px'),
        tooltip = 'Adjust the step in a current/voltage sweep range'
    )
    run_btn = widgets.Button(
        description='Run',
        button_style = '',
        layout = widgets.Layout(width='100px'),
        disabled = False
    )
    stop_btn = widgets.Button(
        description='Stop',
        button_style = 'warning',
        layout = widgets.Layout(width='100px'),
        disabled = True
    )

    step_sweep_link = widgets.dlink((step_selector,'value'),(source_level_slider,'step'))

    limit_label = widgets.Label(f'LIM_{channel} :')
    limitfunc_select = widgets.Dropdown(
        options = ['Power','Primary'],
        value='Power',
        layout = widgets.Layout(width='100px')
    )
    limit_compliance_set = widgets.BoundedFloatText(
        value = 10,
        min = 0.02,
        step = 0.1,
        max = 200,
        layout = widgets.Layout(width = '200px')  
    )

    progress_label = widgets.Label(value="#########")
    progress_bar = widgets.FloatProgress(
        value = 0.0,
        min = 0.0,
        max = 1.0,
        bar_style = "info",
        layout = widgets.Layout(height="30px", width = '295px',align_self='center')
    )

    direct_source_label = widgets.Label(f"Direct source :")
    direct_source_input = widgets.BoundedFloatText(
        value=0.0,
        max = 1,
        min = 0,
        layout=widgets.Layout(width='150px'),
        tooltip='Directly set the source voltage/current level'
    )
    on_btn = widgets.Button(
        description='ON',
        button_style='success',
        layout=widgets.Layout(width='50px')
    )
    off_btn = widgets.Button(
        description='OFF',
        button_style='danger',
        layout=widgets.Layout(width='50px')
    )
    set_btn = widgets.Button(
        description='Set',
        button_style='info',
        layout=widgets.Layout(width='50px')
    )

    source_set_hbox = widgets.HBox([direct_source_input, set_btn, on_btn, off_btn])

    wire_sense_btn = widgets.ToggleButtons(
        options=['2-Wire','4-Wire'],
        disabled = True,
        tooltips=['2-wire local sensing','4-wire remote sensing']
    )
    wire_sense_btn.layout = widgets.Layout(
        width='100px',
        height='auto',
        flex_flow='column',
        display='flex',
        justify_content='center', 
        align_items='center' 
    )
    fast_sweep_btn = widgets.Button(
        description = 'Fast Sweep',
        button_style = 'info',
        layout = widgets.Layout(width = '100px'),
        disabled = True
    )
    mode_select_rb = widgets.RadioButtons(
        options=['IV','VI','VIfourprobe'],
        disabled = True
    )
    mode_select_rb.layout = Layout(width = '200px')
    sense_fs_vbox = widgets.VBox([
        wire_sense_btn,
        fast_sweep_btn,
        mode_select_rb
    ])
    sense_fs_vbox.layout = Layout(width = '200px')
    source_hbox = widgets.HBox([source_range_dropdown,source_mode_dropdown])
    meas_hbox = widgets.HBox([step_selector,run_btn,stop_btn])
    limit_hbox = widgets.HBox([limit_compliance_set,limitfunc_select])

    item_vbox = widgets.VBox([
        source_hbox,
        sweep_hbox,
        meas_hbox,
        limit_hbox,
        progress_bar,
        source_set_hbox
    ])

    label_vbox = widgets.VBox([
        source_label,
        sweep_label,
        step_sel_label,
        limit_label,
        progress_label,
        direct_source_label
    ])
    label_vbox.layout = Layout(width='100px')
    
    channel_hbox = widgets.HBox([label_vbox,item_vbox,sense_fs_vbox])
    channel_hbox.layout = Layout(display='flex', justify_content='space-around')
    
    def on_sourcemode_select(change):
        if change.new == 'Voltage':
            smuX.mode('voltage')
            source_range_dropdown.options = sourcerange_dict_v.keys()
            source_range_dropdown.value = '2V'
        elif change.new == 'Current':
            smuX.mode('current')
            source_range_dropdown.options = sourcerange_dict_i.keys()
            source_range_dropdown.value = '1mA'
    def on_sourcerange_select(change):
        if source_mode_dropdown.value == 'Current':
            range = sourcerange_dict_i[change.new]
            smuX.sourcerange_i(range)
        elif source_mode_dropdown.value == 'Voltage':
            range = sourcerange_dict_v[change.new]
            smuX.sourcerange_v(range)

        unit_mapping = {
            'nA': 1e-9,
            'μA': 1e-6,
            'mA': 1e-3,
            'A': 1,
            'mV': 1e-3,
            'V': 1
        }

        for unit, factor in unit_mapping.items():
            if unit in str(source_range_dropdown.value):
                unit_label.value = unit
                source_level_slider.max = float(round(range / factor))#type:ignore
                source_level_slider.min = -source_level_slider.max
                step_selector.max = source_level_slider.max

                direct_source_input.max = float(round(range / factor))#type:ignore
                direct_source_input.min = -direct_source_input.max
                direct_source_input.step = direct_source_input.max/100
                break
    def on_limitfunc_select(change):
        if change.new == 'Power':
            station.keith.write(f"display.smu{channel.lower()}.limit.func = 1")
            limit_compliance_set.max = 5
            limit_compliance_set.min = 0.001
            limit_compliance_set.value = 5
        elif change.new == 'Primary':
            station.keith.write(f"display.smu{channel.lower()}.limit.func = 0")
            if source_mode_dropdown.value == 'Current':
                limit_compliance_set.max = 200
                limit_compliance_set.min = 0.02
                limit_compliance_set.value = 10
                limit_compliance_set.step = 0.01
            elif source_mode_dropdown.value == 'Voltage':
                limit_compliance_set.max = 1.5
                limit_compliance_set.min = 1e-8
                limit_compliance_set.value = 1e-3
                limit_compliance_set.step = 1e-7
    def on_limit_compliance(change):
        limit = change.new
        if source_mode_dropdown.value == 'Voltage' and limitfunc_select.value == 'Primary':
            smuX.limiti(limit)
        elif source_mode_dropdown.value =='Current' and limitfunc_select.value == 'Primary':
            smuX.limitv(limit)
        elif limitfunc_select.value == 'Power':
            station.keith.write(f"smu{channel.lower()}.source.limitp = {limit}")
    def on_fast_sweep(button):
        if source_mode_dropdown.value == 'Voltage' and (mode_select_rb.value == 'VI' or mode_select_rb.value == 'VIfourprobe'):
            with output : print("Cannot preform a VI sweep on Voltage mode!")
            return
        elif source_mode_dropdown.value == 'Current' and mode_select_rb.value == 'IV':
            with output: print("Cannot preform IV sweep on Current mode!")
            return
        fast_sweep(source_level_slider.lower,source_level_slider.upper,step_selector.value,mode_select_rb.value, smuX, unit_label.value)
        smuX.output('off')
    def on_stop(button):
        global cancel
        cancel = True
        smuX.output('off')
        run_btn.disabled = False
    def show_dialog():
        with output:
            clear_output(wait=True)
            # Check if the value in limit_compliance_set is too low
            if (source_mode_dropdown.value == 'Voltage' and limit_compliance_set.value <= 1e-7) or (source_mode_dropdown.value == 'Current' and limit_compliance_set.value <= 0.02):  # Adjust this threshold as needed
                message = widgets.Label(
                    value="Warning: Setting the compliance too low might damage the device!",
                    style = dict(
                        font_size = '10px',
                        text_color='red'
                    )
                )

                set_widget_disabled_status(channel, True)

                def on_cancel_clicked(b):
                    with output:
                        clear_output(wait=True)
                        print("Operation cancelled!")
                        time.sleep(2)
                        clear_output(wait=False)
                        set_widget_disabled_status(channel, False)
                        stop_btn.disabled = True
                    return
                
                def on_continue_clicked(b):
                    with output:
                        clear_output(wait=True)
                        set_widget_disabled_status(channel, False)
                        Thread(target=run_measurements, args=[output,]).start() 
                    
                
                cancel_button = widgets.Button(
                    description="Cancel", 
                    button_style='info',
                    layout=widgets.Layout(width='100px', height='40px')
                )
                cancel_button.on_click(on_cancel_clicked)
                
                continue_button = widgets.Button(
                    description="Continue",
                    button_style='danger',
                    layout=widgets.Layout(width='100px', height='40px')
                )
                continue_button.on_click(on_continue_clicked)
                
                buttons = widgets.HBox([cancel_button,continue_button])
                buttons.layout = widgets.Layout(width='250px', justify_content='space-around')
                dialog.children = [message, buttons]
                display(dialog)
            else:
                Thread(target=run_measurements, args=[output,]).start() 
    def on_set_sensemode(change):
        if change.new == '4-Wire':
            station.keith.write(f"smu{channel.lower()}.sense = smu{channel.lower()}.SENSE_REMOTE") # If 4-wire sensing is selected
        else:
            station.keith.write(f"smu{channel.lower()}.sense = smu{channel.lower()}.SENSE_LOCAL")  # If 2-wire sensing is selected
    def run_measurements(out : widgets.Output):
        global cancel
        try:
            cancel = False
            run_btn.disabled = True
            run_btn.description = 'Running'
            stop_btn.disabled = False
            with output: print(f"Running sweep on channel {channel}")
            progress_label.value = "Starting operation..."
            progress_bar.bar_style = "info"
            progress_bar.value = 0.0
            
            # two lists V values time values
            sources = np.arange(source_level_slider.lower,source_level_slider.upper,step_selector.value)
            if unit_label.value == 'nA':
                sources = sources/1e9
            elif unit_label.value == 'μA':
                sources = sources/1e6
            elif unit_label.value == 'mA':
                sources = sources/1e3
            elif unit_label.value == 'mV':
                sources = sources/1e3
            
            meas = Measurement(exp = load_or_create_experiment(
                experiment_name=f"{source_mode_dropdown.value}_sweep({wire_sense_btn.value})",
                sample_name="junction_sample"
            ))
            # resistance measurement exp name
            # distance and res value
            buffer = io.StringIO()
            original_stdout = sys.stdout

            sys.stdout = buffer
            smuX.output('on')

            if source_mode_dropdown.value == 'Voltage':
                smuX.measurerange_i(1)
                meas.register_parameter(smuX.volt)
                meas.register_parameter(smuX.curr,setpoints=(smuX.volt,))
                source_setter = smuX.volt
                res_getter = smuX.curr
            elif source_mode_dropdown.value == 'Current':
                smuX.measurerange_v(2)
                meas.register_parameter(smuX.curr)
                meas.register_parameter(smuX.volt,setpoints=(smuX.curr,))
                source_setter = smuX.curr
                res_getter = smuX.volt
            else:
                raise ValueError(f"Unexpected sourcemode_select.value: {source_mode_dropdown.value}")
            
            with meas.run() as datasaver:
                for s in sources:
                    if cancel:  # If cancel flag is set, stop the sweep
                        break
                    source_setter(s) 
                    result = res_getter()
                    datasaver.add_result((source_setter, s),
                                        (res_getter, result))
                    progress_bar.value = (sources.tolist().index(s) + 1) / len(sources)

            datasaver.flush_data_to_database(block=True)
            sys.stdout = original_stdout
            
            # Now, capture the value from buffer and print it in your output widget
            captured_value = buffer.getvalue()
            out.append_stdout(captured_value)
            buffer.close()

            smuX.output('off')

            if cancel:
                progress_label.value = "Operation stopped!"
                progress_bar.bar_style = "danger"
                stop_btn.disabled = True
                run_btn.description = 'Run'
            else:
                progress_bar.bar_style = "success"
                progress_label.value = "Operation complete!"

            time.sleep(1.5)
            progress_bar.value = 0.0
            progress_label.value="#########"
            run_btn.description = 'Run'
            stop_btn.disabled = True
            run_btn.disabled = False
            out.append_stdout("Sweep complete, check database for results and plot")
        
        except Exception as e:
            with output:
                print(f"Error: {e}")
            cancel = True
            smuX.output('off')
            
            progress_label.value = "Operation stopped due to error!"
            progress_bar.bar_style = "danger"
            stop_btn.disabled = True
            run_btn.description = 'Run'

            time.sleep(1.5)
            progress_bar.value = 0.0
            progress_label.value="#########"
            run_btn.description = 'Run'
            run_btn.disabled = False



    def on_set_clicked(button):
        value = direct_source_input.value
        if source_mode_dropdown.value == 'Voltage':
            smuX.volt(value)
        elif source_mode_dropdown.value == 'Current':
            smuX.curr(value)

    def on_on_clicked(button):
        smuX.output('on')

    def on_off_clicked(button):
        smuX.output('off')

    set_btn.on_click(on_set_clicked)
    on_btn.on_click(on_on_clicked)
    off_btn.on_click(on_off_clicked)

    wire_sense_btn.observe(on_set_sensemode,names='value')#type:ignore
    source_mode_dropdown.observe(on_sourcemode_select,names='value')#type:ignore
    source_range_dropdown.observe(on_sourcerange_select,names='value')#type:ignore

    limitfunc_select.observe(on_limitfunc_select,names='value')#type:ignore
    limit_compliance_set.observe(on_limit_compliance,names='value')#type:ignore

    fast_sweep_btn.on_click(on_fast_sweep)

    run_btn.on_click(lambda b: show_dialog())

    stop_btn.on_click(on_stop)


    return channel_hbox


In [40]:

dis_res_label = widgets.Label(value='Distance resistance')
distance_label = widgets.Label(value='Distance : ')
dis_unit_label = widgets.Label(value = '')
distance_text = widgets.FloatText(
    min = 0.000,
    value = 0.1,
    max = 1,
    step = 0.001
)
save_btn = widgets.Button(
    description='Save',
    button_style='success'
)
distance_text.layout = Layout(width = '80px')
ohm_label = widgets.Label(value='Resistance at x distance :')
meas_ohm_btn = widgets.Button(description = 'Measure')
meas_ohm_btn.layout = Layout(align_self='center')
distance_hbox = widgets.HBox([distance_label,distance_text,dis_unit_label])

btns_hbox = widgets.HBox([meas_ohm_btn,save_btn])
dis_res_vbox = widgets.VBox([dis_res_label,distance_hbox,ohm_label,btns_hbox,plot_box])
dis_res_vbox.layout=Layout(border='1px solid black', margin='5px 5px 5px 5px')

sec_meas_hbox = widgets.HBox([dis_res_vbox])
sec_meas_hbox.layout = Layout(display = 'flex')

user_label = widgets.Label(value='Username : ')
user_text = widgets.Text(
    value = 'username',
    tooltip = 'Enter the current user working with the device'
)
user_text.layout = Layout(width='200px',margin='0px 0px 0px 10px')
user_hbox = widgets.HBox([user_label,user_text])
user_hbox.layout = Layout(margin = '10px 10px 10px 10px',align_items='center')

channels_hbox = widgets.HBox()
channels_hbox.layout = Layout(display = 'flex',justify_content='center')
main_ui = widgets.VBox([user_hbox,sec_meas_hbox,output_box])
main_ui.layout = Layout(display='flex', justify_content='space-around')

refresh_btn = widgets.Button(
    description='Refresh',
    button_style='success',
    tooltip='Refreshes the data overview with the latest expirements'
)
refresh_btn.layout = Layout(width = '100px',height = '40px')
export_btn = widgets.Button(
    description='Export data',
    button_style='warning',
    tooltip='Exports all of the datasets to a .csv file, by default every dataset is exported to the username_database_db folder'
)
export_btn.layout = Layout(width = '100px',height = '40px')
data_btns = widgets.HBox([refresh_btn,export_btn])

data_overview = qcw.experiments_widget()
data_vbox = widgets.VBox([data_overview.children[1], data_overview.children[2]]) #type:ignore

database_vbox = widgets.VBox([data_btns,data_vbox])
tab_widget = widgets.Tab(
    children=[main_ui,database_vbox],
    titles=['Main UI','Data Overview']
)


In [41]:
def on_disconnect(button):
    user_label.value = "Username : "
    user_label.style = dict()
    user_hbox.children = [user_label,user_text]
    disconnect_device()
    connect_btn.disabled = False
    connect_btn.button_style = ''
    disconnect_btn.disabled = True
    channels_hbox.children = []
    main_ui.children = [user_hbox, sec_meas_hbox, output_box]
    set_widget_disabled_status(sec_meas_hbox,True)
def on_connect(button):
    global data_overview
    global channel_A_hbox
    global channel_B_hbox
    global channels_initialized
    if user_text.value == "" or user_text.value == "username":
        with output:
            print("Please enter a valid username!!")
        return
    else:
        username = user_text.value
        db_path = get_database_path(username)
        if db_path is None:
            db_path = add_user(username)

        with open(USER_JSON_PATH, 'r') as json_file:
            data = json.load(json_file)
        user_data = data.get(username, None)
        if user_data:
            config_path = user_data["config_path"]
            qc.config.load_config(config_path)
        
        if connect_device():
            channel_A_hbox = setup_channel('A')
            channel_B_hbox = setup_channel('B')
            channels_initialized = True
            channels_hbox.children = [channel_A_hbox, channel_B_hbox]
            main_ui.children = [user_hbox, channels_hbox, sec_meas_hbox, output_box]
            
            set_widget_disabled_status(sec_meas_hbox,False)
            set_widget_disabled_status(channel_A_hbox, False)
            set_widget_disabled_status(channel_B_hbox, True)
            disconnect_btn.disabled = False
            connect_btn.disabled = True
            connect_btn.button_style = 'success'
            with output: print(f"User {username} succesfully connected and created a database at : {db_path}")

            qcd.initialise_or_create_database_at(db_path)
            db = qcd.connect(db_path)
            data_overview = qcw.experiments_widget(db = db_path,sort_by="timestamp")
            data_vbox.children = [data_overview.children[1], data_overview.children[2]]#type:ignore

            tab_widget.children = [main_ui, database_vbox]
            
            user_label.value = f"Welcome {user_text.value}"
            user_label.style = dict(font_weight = 'bold', font_size = '14px')
            user_hbox.children = [user_label]

def on_exit_key(button):
    station.keith.exit_key()
def on_clear(button):
    with output: clear_output(wait=False)

def on_refresh(button):
    data_overview = qcw.experiments_widget(sort_by="timestamp")
    data_vbox.children = [data_overview.children[1], data_overview.children[2]]#type:ignore
    with output: print("Database refreshed. In case of newer experiments still not showing, please disconnect then connect with your username!")
def on_export(button):
    all_experiments = experiments()
    
    all_datasets = []
    for exp in all_experiments:
        # Append datasets from each experiment to our list
        all_datasets.extend(exp.data_sets())
   
    for dataset in all_datasets:
        export_path = os.path.join(EXPORTS_DIR,f"{user_text.value}_dataset_{dataset.run_id}")
        dataset.export("csv",path=export_path)
        
    with output : print("Data has been exported to the 'Exports' file!")
    
def adjust_channels(change):
    global channel_A_hbox, channel_B_hbox
    if change.new == "SMUA":
        set_widget_disabled_status(channel_A_hbox, False)
        set_widget_disabled_status(channel_B_hbox, True)
        station.keith.write(f"display.screen = {display_options[change.new]}")
    elif change.new == "SMUB":
        set_widget_disabled_status(channel_A_hbox, True)
        set_widget_disabled_status(channel_B_hbox, False)
        station.keith.write(f"display.screen = {display_options[change.new]}")
    elif change.new == "SMUA & SMUB":
        set_widget_disabled_status(channel_A_hbox, False)
        set_widget_disabled_status(channel_B_hbox, False)
        station.keith.write(f"display.screen = {display_options[change.new]}")

def measure_resistance(output: widgets.Output):
    try:
        # Get the distance from the input widget
        distance = float(str(distance_text.value))
        # Measure resistance
        station.keith.smua.output('on')
        resistance = station.keith.smua.res()
        station.keith.smua.output('off')
        ohm_label.value = f"Resistance at {distance_text.value} {dis_unit_label.value} : {resistance}"
        
        # Update data lists
        distances.append(distance)
        resistances.append(resistance)
        
        # Update plot
        update_plot()
    except Exception as e:
        output.append_stdout(f"Error during measurement: {e}")

def on_measure(button):
    measure_resistance(output)

def on_save(button):
    distance_param = Parameter('distance', label='Distance', unit=str(dis_unit_label.value))
    resistance_param = Parameter('resistance', label='Resistance', unit='Ohm')

    # Set up the measurement
    meas = Measurement(exp = load_or_create_experiment(
                experiment_name=f"Resistance_measurement({channel_A_hbox.children[1].children[0].children[1].value}_Source)",#type:ignore
                sample_name="Resistance_vs_distance"
            ))
    meas.register_parameter(distance_param)
    meas.register_parameter(resistance_param)
    with output:
        with meas.run() as datasaver:
            for d, r in zip(distances, resistances):
                datasaver.add_result((distance_param, d), (resistance_param, r))
            datasaver.flush_data_to_database()

def on_distance_change(change):
    if change.new >= 1e-3 and change.new < 1e-2:
        dis_unit_label.value = 'mm'
    elif change.new >= 0.01 and change.new < 0.1:
        dis_unit_label.value = 'cm'
    elif change.new >= 0.1 and change.new < 1:
        dis_unit_label.value = 'dm'
    else:
        dis_unit_label.value = 'm'
distance_text.observe(on_distance_change,names='value')#type:ignore

save_btn.on_click(on_save)
meas_ohm_btn.on_click(on_measure)
display_dropdown.observe(adjust_channels,names='value')#type:ignore

refresh_btn.on_click(on_refresh)
export_btn.on_click(on_export)

exit_btn.on_click(on_exit_key)

clear_btn.on_click(on_clear)

connect_btn.on_click(on_connect)
disconnect_btn.on_click(on_disconnect)


In [42]:
set_widget_disabled_status(channel_B_hbox,True)
set_widget_disabled_status(sec_meas_hbox,True)
display(HTML("""
<style>
    .output_area {
        word-wrap: break-word;
    }
</style>
"""))
display(tab_widget)